# Structured Data Analysis with Spark SQL

Big Data is not only raw text files, a lot of datasets available are structured as table and even raw text files have a underlying structure. Spark SQL allow to query structured datasets, relational database and also provide an API to structure data.

## Table of Content

1. [Initialization](#1.-Initialization)  
  1.1 [Spark](#1.1-Spark)  
  1.2 [Spark SQL](#1.2-Spark-SQL)
2. [Dataframe](#2.-Dataframe)  
  2.1 [Reading Data](#2.1-Reading-Data)  
  2.2 [Structuring Data](#2.2-Structuring-Data)  
  2.3 [Creating a Dataframe](#2.3-Creating-a-Dataframe)  
  2.4 [Registering a Table](#2.4-Registering-a-Table)  
  2.5 [Querying Data](#2.5-Querying-Data)  
  2.6 [Aggregating Results](#2.6-Aggregating-Results)
3. [Writing Results to Disk](#3.-Writing-Results-to-Disk)  
  3.1 [Apache Parquet](#3.1-Apache-Parquet)  
  3.2 [Other Formats](#3.2-Other-Formats)    
4. [Reading the Dataframe from Storage](#4.-Reading-the-Dataframe-from-Storage)  
  4.1 [Reading CSV Files](#4.1-Reading-CSV-Files)
4. [Ending Spark SQL Analysis](#5.-Ending-Spark-SQL-Analysis)
6. [Recap](#6.-Recap)
7. [References](#References)

## List of Exercises

1. [Exercise 1: Initialize Spark](#Exercise-1)
2. [Exercise 2: Create an RDD](#Exercise-2)
3. [Exercise 3: Count Elements](#Exercise-3)
4. [Exercise 4: Transform an RDD](#Exercise-4)
5. [Exercise 5: Validate Transformation](#Exercise-5)
6. [Exercise 6: Type RDD fields](#Exercise-6)
7. [Exercise 7: Create a Dataframe](#Exercise-7)
8. [Exercise 8: Display a Dataframe](#Exercise-8)
9. [Exercise 9: End the Context](#Exercise-9)

## 1. Initialization

### 1.1 Spark
#### Exercise 1

Import the necessary Python module(s) and create a Spark context. 

**Warning**, verify if there exists a context and handle possible exceptions.

In [1]:
import pyspark
try:
    sc = pyspark.SparkContext()
except ValueError:
    pass

### 1.2 Spark SQL

We can now import the components that we need to analyze structured data from Spark SQL module `pyspark.sql`.
* `SQLContext`: Main entry point for Spark SQL functionality. It will be used to create Dataframe.

In [2]:
from pyspark.sql import SQLContext

Spark SQL Context requires a Spark Context as sole argument.

In [3]:
sqlContext = SQLContext(sc)

## 2. Dataframe

Textbook definition:    

> A data frame is a table, or two-dimensional array-like structure, in which each column contains measurements on one variable, and each row contains one case.

In Spark, a dataframe is a distributed collection of data grouped into named columns. It is equivalent to a relational table.

### 2.1 Reading Data

#### Exercise 2

Create an RDD with the dataset we used previously `data/pagecounts`.

In [4]:
rdd = sc.textFile('data/pagecounts/*.gz')

#### Exercise 3

Count the number of elements in the RDD.

In [5]:
rdd.count()

11268665

#### Exercise 4

Transform the previous RDD into a second one where each field originally separated by white spaces are now elements of a list.

In [6]:
rdd = rdd.map(str.split)

#### Exercise 5

To validate the transformation, display the first 8 elements of the RDD.

In [7]:
rdd.take(8)

[['aa.b', 'Main_Page', '3', '16288'],
 ['aa.b', 'Special:RecentChangesLinked/User:Az1568', '1', '5745'],
 ['aa.b', 'Special:Recentchangeslinked/User:Az1568', '1', '1013'],
 ['aa.b', 'Special:Statistics', '1', '840'],
 ['aa.b', 'Template:Delete', '1', '26601'],
 ['aa.d', 'Main_Page', '2', '5442'],
 ['aa', 'Main_Page', '4', '19955'],
 ['aa', 'User:Alecs.bot', '1', '13388']]

#### Exercise 6

As you can see, the third and fourth fields are numbers represented as text. Transform the RDD in order to convert these strings to `int`.

In [8]:
rdd = rdd.map(lambda x: (x[0], x[1], int(x[2]), int(x[3])))

### 2.2 Structuring Data

Our original dataset is strictly text, we now want to give a structure that is define field name and type.

To define our structure, we use Spark SQL data types that are defined in [`pyspark.spl.types`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types). We import a small subset of type that we need `LongType` and `StringType`.

In [9]:
from pyspark.sql.types import LongType, StringType

Spark SQL also provides two types to defines dataframe structure:
- `StructType`: Data type representing a row of a dataframe.
- `StructField`: Data type representing a field of a row. It is mainly defined by a name and a type.

In [10]:
from pyspark.sql.types import StructType, StructField

Using all these classes, we can define our data schema. The order in the list must correspond to the order in our dataset.

In [11]:
schema = StructType([StructField('lang',    StringType()), 
                     StructField('name',    StringType()), 
                     StructField('request', LongType()), 
                     StructField('size',    LongType())])

### 2.3 Creating a Dataframe

To create a dataframe, we simply need to invoke the `createDataFrame()` method of our Spark SQL Context and provide it an RDD and our data structure (or schema). 

#### Exercise 7

Replace `<FILL IN>` in the following cell by the proper RDD for `data/pagecounts`, and the second `<FILL IN>` by the method call to persist the dataframe in memory. **Hint**: it is the same method for RDDs.

In [12]:
dfPageCounts = sqlContext.createDataFrame(rdd, schema).cache()

We can then manipulate the dataframe using its [`pyspark.sql.DataFrame` API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame).

We can, for example, show the first lines of the dataframe as a table in ASCII.

In [13]:
dfPageCounts.show()

+----+--------------------+-------+-----+
|lang|                name|request| size|
+----+--------------------+-------+-----+
|aa.b|           Main_Page|      3|16288|
|aa.b|Special:RecentCha...|      1| 5745|
|aa.b|Special:Recentcha...|      1| 1013|
|aa.b|  Special:Statistics|      1|  840|
|aa.b|     Template:Delete|      1|26601|
|aa.d|           Main_Page|      2| 5442|
|  aa|           Main_Page|      4|19955|
|  aa|      User:Alecs.bot|      1|13388|
|  ab|%D0%90%D0%B8%D0%B...|      1|  465|
|  ab|%D0%98%D1%85%D0%B...|      1|16098|
|  ab|%D0%9C%D0%B8%D1%8...|      1| 8829|
|  ab|%D0%A1%D0%BB%D1%8...|      1|20699|
|  ab|%D0%A1%D0%BB%D1%8...|      1|21104|
|  ab|%D0%A1%D0%BB%D1%8...|      1|21243|
|  ab|%D0%A1%D0%BB%D1%8...|      2|42212|
|  ab|%D0%A1%D0%BB%D1%8...|      1|20696|
|  ab|%D0%A1%D0%BB%D1%8...|      1|23882|
|  ab|%D0%A1%D0%BB%D1%8...|      1|22220|
|  ab|%D0%A1%D0%BB%D1%8...|      2|43088|
|  ab|%D0%A1%D0%BB%D1%8...|      1|21408|
+----+--------------------+-------

### 2.4 Registering a Table

In order to query our dataframe with SQL, we need to register it as a table in the Spark SQL context.

In [14]:
dfPageCounts.registerTempTable("page_table")

### 2.5 Querying Data

We are now able to interogate our data using the Structured Query Language (SQL). The following query request the top 10 most requested page in spanish.

In [15]:
df = sqlContext.sql("SELECT name, request "
                    "FROM page_table "
                    "WHERE lang='es' "
                    "ORDER BY request DESC "
                    "LIMIT 10")

A query on a dataframe is a Spark transformation. Therefore, to compute the result, we need to call an action. 

#### Exercise 8 

Call the right method to display the dataframe resulting from the preceding query.

In [16]:
df.show()

+--------------------+-------+
|                name|request|
+--------------------+-------+
|   Wikipedia:Portada|  21157|
|   Wikipedia:Portada|  17867|
|   Wikipedia:Portada|  16078|
|       Gripe_porcina|   9847|
|      Special:Search|   9746|
|       Gripe_porcina|   9660|
|       Gripe_porcina|   8356|
|      Special:Search|   8076|
|      Special:Search|   7888|
|Brote_de_gripe_po...|   4664|
+--------------------+-------+



### 2.5.1 SQL 101

We can decompose the preceding query in keywords:

#### SELECT

Indicate which variable we want to collect. The name of variable have been defined when structuring our data in [section 2.2](#2.2-Structuring-Data).

#### FROM

Indicate the source of data. The name of the table has been defined in [section 2.4](#2.4-Registering-a-table).

#### WHERE

Filter the entries based on predicates in function of the variables. 

#### ORDER BY [...] DESC

Indicate we wish to order the resulting dataframe in function of a certain variable, in a certain order. 

#### LIMIT N 

Return only a subset of entries.

### 2.5.2 SQL as an API

[Dataframe API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame) includes methods that are named after SQL keywords. These methods can be used instead of the query language. However, the order does not match exactly as the query is a single statement, the API will return a dataframe after each method call. These calls can be chained to build a similar pipeline.

The following example extract and show the 10 most requested pages in English on Wikipedia.

In [17]:
dfPageCounts.where("lang = 'en'")\
            .select("name", "request")\
            .orderBy("request", ascending=False)\
            .limit(10).show()

+--------------+-------+
|          name|request|
+--------------+-------+
|    404_error/| 626973|
|    404_error/| 591572|
|    404_error/| 565808|
|     Main_Page| 256293|
|Special:Search| 244353|
|     Main_Page| 242325|
|Special:Search| 239859|
|     Main_Page| 232439|
|Special:Search| 231514|
|Special:Random|  61915|
+--------------+-------+



### 2.6 Aggregating Results

If we look back at the preceding results, we realise that some names appear multiple time. The reason is that we have omitted to sum the number of requests per page. We need to aggregate the page entries by name and sum the requests.

To do this, we indicate in our query to sum the request (`SUM(request)`) for entries grouped by name (`GROUP BY NAME`).

In [18]:
sqlContext.sql("SELECT name, SUM(request)"
               "FROM page_table "
               "WHERE lang='es' "
               "GROUP BY name "
               "ORDER BY SUM(request) DESC "
               "LIMIT 10").show()

+--------------------+-----+
|                name|  _c1|
+--------------------+-----+
|   Wikipedia:Portada|55102|
|       Gripe_porcina|27863|
|      Special:Search|25710|
|Brote_de_gripe_po...|12896|
|            Pandemia| 9990|
|D%C3%ADa_del_Trabajo| 4719|
|  Especial:Aleatoria| 3245|
|Archivo:H1N1_map.svg| 2876|
|       Peste_porcina| 2718|
|Copa_Libertadores...| 2718|
+--------------------+-----+



#### Aggregating Using the API

In [19]:
dfPageCounts.where("lang = 'en'")\
            .select("name", "request")\
            .groupBy('name')\
            .agg({'request' : 'sum'})\
            .orderBy("sum(request)", ascending=False)\
            .limit(10).show()

+--------------------+------------+
|                name|sum(request)|
+--------------------+------------+
|          404_error/|     1784353|
|           Main_Page|      731057|
|      Special:Search|      715726|
|      Special:Random|      165826|
|     Swine_influenza|      145112|
|  Special:Randompage|      118477|
|2009_swine_flu_ou...|       47441|
|           Swine_flu|       29862|
|            Pandemic|       16965|
|                Wiki|       16867|
+--------------------+------------+



## 3. Writing Results to Disk

To avoid structuring our data each time, we can save the resulting dataframe to disk. This will preserve the schema and the data order.

### 3.1 Apache Parquet

[Apache Parquet](https://parquet.apache.org/) is a columnar storage format available to any project in the Hadoop ecosystem, regardless of the choice of data processing framework, data model or programming language. The format is fairly popular with the Spark community as it is efficient and easy to use.

In [20]:
dfPageCounts.write.parquet("data/pagecounts.parquet")

### 3.2 Other Formats

Dataframe can also be written as JSON file: 

In [21]:
dfPageCounts.write.json('data/pagecounts.json')

It can also save the dataframe as a table in [Apache Hive](http://hive.apache.org/) or a database. 

## 4. Reading the Dataframe from Storage

A new Dataframe can be created by reading back from a file in one of the format we mentionned.

For example, to create a dataframe from a Parquet file:

In [22]:
pagecount_parq = sqlContext.read.parquet("data/pagecounts.parquet")
pagecount_parq.show()

+----+--------------------+-------+-----+
|lang|                name|request| size|
+----+--------------------+-------+-----+
|aa.b|           Main_Page|      3|16288|
|aa.b|Special:RecentCha...|      1| 5745|
|aa.b|Special:Recentcha...|      1| 1013|
|aa.b|  Special:Statistics|      1|  840|
|aa.b|     Template:Delete|      1|26601|
|aa.d|           Main_Page|      2| 5442|
|  aa|           Main_Page|      4|19955|
|  aa|      User:Alecs.bot|      1|13388|
|  ab|%D0%90%D0%B8%D0%B...|      1|  465|
|  ab|%D0%98%D1%85%D0%B...|      1|16098|
|  ab|%D0%9C%D0%B8%D1%8...|      1| 8829|
|  ab|%D0%A1%D0%BB%D1%8...|      1|20699|
|  ab|%D0%A1%D0%BB%D1%8...|      1|21104|
|  ab|%D0%A1%D0%BB%D1%8...|      1|21243|
|  ab|%D0%A1%D0%BB%D1%8...|      2|42212|
|  ab|%D0%A1%D0%BB%D1%8...|      1|20696|
|  ab|%D0%A1%D0%BB%D1%8...|      1|23882|
|  ab|%D0%A1%D0%BB%D1%8...|      1|22220|
|  ab|%D0%A1%D0%BB%D1%8...|      2|43088|
|  ab|%D0%A1%D0%BB%D1%8...|      1|21408|
+----+--------------------+-------

### 4.1 Reading CSV Files

Datasets are often found as CSV files. Unfortunately, Spark does not provide builtin methods to read CSV. However, [Databricks](https://www.databricks.com/), the main company behind Spark developement, provides [spark-csv](https://github.com/databricks/spark-csv), an addon library for parsing and querying CSV data with Apache Spark, for Spark SQL and DataFrames. 

The addon library is already installed and linked to for this workshop. One wanting to install it himself could consult the following StackOverflow answer to know how : [How to add any new library like spark-csv in Apache Spark prebuilt version](http://stackoverflow.com/questions/30757439/how-to-add-any-new-library-like-spark-csv-in-apache-spark-prebuilt-version).

The following line read the employee dataset of a store as a Dataframe and display its content. The dataset is a Tab Separated Values (TSV) file, therefore we need to specify the `delimiter` to Spark as `\t`, the tab character.

In [23]:
df = sqlContext.read.format('com.databricks.spark.csv')\
              .options(header='true', inferschema='true', delimiter='\t')\
              .load('data/store/employees.tsv')
df.show()

+----------+---------+---------+---------+-----------+--------------------+
|EmployeeID| LastName|FirstName|BirthDate|      Photo|               Notes|
+----------+---------+---------+---------+-----------+--------------------+
|         1|  Davolio|    Nancy|12/8/1968| EmpID1.pic|Education include...|
|         2|   Fuller|   Andrew|2/19/1952| EmpID2.pic|Andrew received h...|
|         3|Leverling|    Janet|8/30/1963| EmpID3.pic|Janet has a BS de...|
|         4|  Peacock| Margaret|9/19/1958| EmpID4.pic|Margaret holds a ...|
|         5| Buchanan|   Steven| 3/4/1955| EmpID5.pic|Steven Buchanan g...|
|         6|   Suyama|  Michael| 7/2/1963| EmpID6.pic|Michael is a grad...|
|         7|     King|   Robert|5/29/1960| EmpID7.pic|Robert King serve...|
|         8| Callahan|    Laura| 1/9/1958| EmpID8.pic|Laura received a ...|
|         9|Dodsworth|     Anne| 7/2/1969| EmpID9.pic|Anne has a BA deg...|
|        10|     West|     Adam|9/19/1928|EmpID10.pic|        An old chum.|
+----------+

## 5. Ending Spark SQL Analysis

Spark SQL's context do not need to be terminated prior to leaving the notebook.

#### Exercise 9

Terminate the Spark Context.

In [24]:
sc.stop()

## 6. Recap

In this notebook, we put in practice and learned about the following parts of 
**[Python Spark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html)**:
1. Import Spark SQL Python module: 
**[`import pyspark.sql`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html)**
2. Create a Spark SQLContext:
**[`pyspark.sql.SQLContext()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext)**
3. Create an RDD from text files:
**[`SparkContext.textFile(path)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.textFile)**
4. Count the number of elements in a RDD: 
**[`Rdd.count()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.count)**
5. Apply a transformation on each element of a RDD:
**[`RDD.map(func)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map)**
6. Take a the first *num* elements from an RDD: 
**[`Rdd.take(num)`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take)**
7. Structure and type data fields: **[`pyspark.sql.types`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.types)**
8. Create a dataframe from an RDD: **[`SQLContext.createDataFrame(RDD)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.createDataFrame)**
9. Print the first *n* rows of a dataframe: **[`Dataframe.show(n)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.show)**
10. Register a dataframe as a temporary table: **[`DataFrame.registerTempTable(name)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.registerTempTable)**
11. Query a context's registered table: **[`SQLContext.sql(name)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext.sql)**
12. Use the SQL API of a dataframe: **[`DataFrame`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame)**
13. Write a dataframe as Parquet file: **[`DataFrame.write.parquet(path)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter.parquet)**
14. Write a dataframe as JSON file: **[`DataFrame.write.json(path)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameWriter.json)**
15. Read a dataframe from a Parquet file: **[`DataFrame.read.parquet(path)`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.parquet)**  
16. Read a dataframe from a CSV file: **[`DataFrame.read.format('com.databricks.spark.csv')`](https://github.com/databricks/spark-csv#python-api)**
10. End the SparkContext:
**[`SparkContext.stop()`](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.stop)**

## 7. References

* [Berkeley AmpCamp 5 - Data Exploration Using Spark SQL](http://ampcamp.berkeley.edu/5/exercises/data-exploration-using-spark-sql.html)
* [edX - Introduction to Big Data with Apache Spark](https://www.edx.org/course/introduction-big-data-apache-spark-uc-berkeleyx-cs100-1x)
* [edX - Introduction to Big Data with Apache Spark (Github repo)](https://github.com/spark-mooc/mooc-setup)